In [1]:
import argopy
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from erddapy import ERDDAP
import datetime as dt

In [2]:
# Get the ARGO data via ERDDAP (loop from 1997 to 2023)

sst_yrs = {}

for year in np.arange(2023,2024):
    print(year)
    e = ERDDAP(
        server="https://www.ifremer.fr/erddap",
        protocol="tabledap"
    )
    
    e.response = "csv"
    e.dataset_id = "ArgoFloats"
    e.constraints = {
        "time>=": f"{year}-01-01T00:00:00Z",
        "time<=": f"{year}-12-31T00:00:00Z",
        "latitude>=": 0,
        "latitude<=": 90,
        "longitude>=": -99,
        "longitude<=": 13,
    }
    e.variables = [
        "time", "latitude", "longitude", "temp", "pres"
    ]
    
    df = e.to_pandas()
    df = df.rename(columns={'time (UTC)': 'time',
           'latitude (degrees_north)': 'latitude',
           'longitude (degrees_east)': 'longitude',
           'temp (degree_Celsius)': 'temp',
           'pres (decibar)': 'pres'})
    sst = df[df.pres <= 10]
    sst.loc[:,'time'] = sst.time.apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").date())
    sst = sst.drop(columns='pres')
    sst_yrs[str(year)] = sst.set_index('time')
    sst_yrs[str(year)].to_csv(f'../../data/argo_{year}.csv',)

2023
